In [1]:
import numpy as np
import scipy.optimize as opt

In [2]:
f_raw = open('quebec.dat','rb')
sample_size = 3090
cate_list = [0,3,4,5,8]
cate_name = ['sector','conv_year','house_type','constr_year','own_rent']
cate_data = np.ndarray(shape=(sample_size,len(cate_list)),dtype=np.int64)
shared_list = [1,6,7,9,10,11]
shared_data = np.ndarray(shape=(sample_size,len(shared_list)),dtype=np.float64)
num_labels = 9
num_individual = 4
individual_data = np.ndarray(shape=(num_labels,sample_size,num_individual),dtype=np.float64)
label_i = np.ndarray(shape=(sample_size),dtype=np.int64)
label = np.zeros(shape=(num_labels,sample_size),dtype=np.float64)
count = 0

for line in f_raw:
    if count > 0:
        data = line.strip().split("\t")
        # sector, hdd, choice, conv_year, house_type, constr_year, nb_rooms, nb_pers,
        #    own_rent, surface, age, income = data[:12]
        # op_cost = data[12:21]
        # fix_cost = data[21:30]
        # cost_inc = data[30:39]
        # avail = data[39:48]   
        for i in xrange(len(cate_list)):
            cate_data[count-1,i] = int(data[cate_list[i]])
        for i in xrange(len(shared_list)):
            shared_data[count-1,i] = float(data[shared_list[i]])
        for i in xrange(num_individual):
            for j in xrange(num_labels):
                individual_data[j,count-1,i] = float(data[12+i*num_labels+j])
        label_i[count-1] = int(data[2])-1
        label[int(data[2])-1,count-1] = 1.0
    count += 1
    
f_raw.close()

num_vari = len(shared_list)+num_individual+1
numer_data = np.ones(shape=(num_labels,sample_size,num_vari),dtype=np.float64)
numer_data[:,:,(len(shared_list)+1):] = individual_data
for i in xrange(num_labels):
    numer_data[i,:,1:(len(shared_list)+1)] = shared_data

In [37]:
def likelihood(w, data, label):
    num_labels, sample_size, num_vari = data.shape
    util = np.zeros((num_labels,sample_size),np.float64)
    for i in xrange(num_labels):
        util[i,:] = np.dot(data[i,:,:],w[i*num_vari:(i+1)*num_vari])
    util = np.exp(util - util.max(axis=0))
    prob = util / util.sum(axis=0)
    prob = np.sum(prob * label,axis=0)
    out = np.sum(-np.log(prob))
    return out

def l_gradient(w, data, label):
    num_labels, sample_size, num_vari = data.shape
    out = np.zeros((num_labels,num_vari),np.float64)
    util = np.zeros((num_labels,sample_size),np.float64)
    for i in xrange(num_labels):
        util[i,:] = np.dot(data[i,:,:],w[i*num_vari:(i+1)*num_vari])
    util = np.exp(util - util.max(axis=0))
    prob = util / util.sum(axis=0)
    for i in xrange(num_labels):
        out[i,:] = np.sum(data[i,:,:].T * (prob[i,:] - label[i,:]),axis=1)
    out = np.reshape(out,(num_vari*num_labels))
    if np.dot(out,out) > 1e-8:
        out = out / np.sqrt(np.dot(out,out))
    return out

def p_gradient(w, data, label, epsilon):
    num_labels, sample_size, num_vari = data.shape
    grad_temp = np.zeros((sample_size,num_labels,num_vari),np.float64)
    out = np.ndarray(shape=(sample_size,num_labels*num_vari),dtype=np.float64)
    util = np.zeros((num_labels,sample_size),np.float64)
    for i in range(num_labels):
        util[i,:] = np.dot(data[i,:,:],w[i*num_vari:(i+1)*num_vari])
    util = np.exp(util - util.max(axis=0))
    prob = util / util.sum(axis=0)
    for i in xrange(num_labels):
        grad_temp[:,i,:] = (data[i,:,:].T * (prob[i,:] - label[i,:])).T
    for n in xrange(sample_size):   
        grad = np.reshape(grad_temp[n,:,:],(num_vari*num_labels))
        if np.dot(grad,grad) > 1e-8:
            grad = grad / np.sqrt(np.dot(grad,grad))
        out[n,:] = grad
    return out

In [39]:
weight_init = np.zeros(num_vari*num_labels,np.float64)
loss = likelihood(weight_init, numer_data, label)
delta_loss = loss
print 'initial: ',loss
delta_bound = delta_loss * 1e-3
epsilon = 1.0 / np.sqrt(sample_size) * 1e0
class_size = 1
class_max = 10
data_list = [numer_data]
weight_list = [weight_init]
label_list = [label]
category_list = [cate_data]
iter_count = 0

while (delta_loss > delta_bound) and (class_size < class_max):
    # first step: calculate MLE for each class
    loss_old = loss
    loss = 0
    print 'iteration ',iter_count,':'
    for i in xrange(class_size):
        res = opt.minimize(likelihood,weight_list[i],jac = l_gradient,
                                       args = (data_list[i],label_list[i]))
        weight_list[i] = res.x
        temp_loss = likelihood(weight_list[i],data_list[i],label_list[i])
        loss += temp_loss
        print 'class ',i,' loss:',temp_loss
    print 'total loss:',loss
    delta_loss = loss_old - loss
    
    # second step: calculate pointwise gradient (transformed)
    # third step: classification with categorical data
    max_i = -1
    max_rate = 0
    for i in xrange(class_size):
        category_label = p_gradient(weight_list[i],data_list[i],label_list[i],epsilon)
        category_data = category_list[i]
        category_variable_size = category_data.shape[1]
        # cate_data with shape (n,m_c); cate_label with shape (n,m_w)
        max_v = np.sum(np.abs(np.sum(category_label,axis=0)))
        max_c = -1
        max_t = -1
        for c in xrange(category_variable_size):
            type_set = np.unique(category_data[:,c])
            for t in type_set:
                v = 0
                v += np.sum(np.abs(np.sum(category_label[category_data[:,c]<=t,:],axis=0)))
                v += np.sum(np.abs(np.sum(category_label[category_data[:,c]>t,:],axis=0)))
                if v > max_v:
                    max_v = v
                    max_c = c
                    max_t = t
        
        rate = float(max_v) - float(np.sum(np.abs(np.sum(category_label,axis=0))))
        if rate > max_rate:
            max_rate = rate
            max_i = i
    
    if max_i >= 0:
        category_data = category_list[max_i]
        ind_1 = category_data[:,max_c]<=max_t
        ind_2 = category_data[:,max_c]>max_t
        if (np.sum(ind_1)>0) and (np.sum(ind_2)>0):
            class_size += 1
            
            data_temp = data_list[max_i]
            data_list.append(data_temp[:,ind_1,:])
            data_list.append(data_temp[:,ind_2,:])
            del data_list[max_i]
            del data_temp
            
            weight_list.append(weight_list[max_i])
            weight_list.append(weight_list[max_i])
            del weight_list[max_i]
            
            label_temp = label_list[max_i]
            label_list.append(label_temp[:,ind_1])
            label_list.append(label_temp[:,ind_2])
            del label_list[max_i]
            del label_temp
            
            category_temp = category_list[max_i]
            category_list.append(category_temp[ind_1,:])
            category_list.append(category_temp[ind_2,:])
            del category_list[max_i]
            del category_temp
            
            print 'iteration ',iter_count,': category=',cate_name[max_c],', boundary value=',max_t
        else:
            delta_loss = 0
    else:
        delta_loss = 0
    
    iter_count += 1

initial:  6789.42394397
iteration  0 :
class  0  loss: 1361.97543072
total loss: 1361.97543072
iteration  0 : category= constr_year , boundary value= 7
iteration  1 :
class  0  loss: 560.960256356
class  1  loss: 498.664979062
total loss: 1059.62523542
